In [3]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#imprt libraries
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import math
import re
import random
import string
import time
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import random
from datasets import load_dataset
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr     

In [6]:
train_es = load_dataset("stsb_multi_mt", name="es", split="train")
val_es = load_dataset("stsb_multi_mt", name="es", split="dev")
test_es = load_dataset("stsb_multi_mt", name="es", split="test")


In [7]:
train_en = load_dataset("stsb_multi_mt", name="en", split="train")
val_en = load_dataset("stsb_multi_mt", name="en", split="dev")
test_en = load_dataset("stsb_multi_mt", name="en", split="test")


In [8]:
trainEN_sen1 = []
trainES_sen2 = []
testEN_sen1 = []
testES_sen2 = []
valEN_sen1 = []
valES_sen2 = []
train_score = []
test_score = []
val_score = []

# train
for length in range (len(train_es)):
  trainEN_sen1.append(train_es[length]["sentence1"].lower())
  train_score.append(train_es[length]["similarity_score"])


for length in range (len(train_en)):
  trainES_sen2.append(train_en[length]["sentence2"].lower())

# val
for length in range (len(val_es)):
  valEN_sen1.append(val_es[length]["sentence1"].lower())
  val_score.append(val_es[length]["similarity_score"])


for length in range (len(val_en)):
  valES_sen2.append(val_en[length]["sentence2"].lower())

# test
for length in range (len(test_es)):
  testEN_sen1.append(test_es[length]["sentence1"].lower())
  test_score.append(test_es[length]["similarity_score"])


for length in range (len(test_en)):
  testES_sen2.append(test_en[length]["sentence2"].lower())


In [28]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import numpy as np
from googletrans import Translator
import gensim.downloader as api


In [33]:
# Define the model repo
model_name = "Helsinki-NLP/opus-mt-en-es" 


# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [34]:
# Define the input sentence in Spanish
source_text = "hi how are you?"

# Tokenize the input sentence
inputs = tokenizer(source_text, return_tensors="pt")

# Generate translation from the input sentence
outputs = model.generate(**inputs)

# Decode the generated translation into text
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated translation
print(decoded_output)


Hola, ¿cómo estás?


In [35]:
print(len(testES_sen2))

1379


In [38]:
outs = []
i=0
for sent in testES_sen2[:100]:
    # Tokenize the input sentence
    inputs = tokenizer(source_text, return_tensors="pt")

    # Generate translation from the input sentence
    outputs = model.generate(**inputs)

    # Decode the generated translation into text
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    outs.append(decoded_output)
    i +=1
    if i%5 == 0:
      print(i)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100


In [39]:
print(len(testES_sen2[:100]))
print(len(outs))
print(len(testEN_sen1[:100]))

100
100
100


In [40]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [41]:
%cd /content/drive/My Drive/NLP/PreTrained_models


/content/drive/My Drive/NLP/PreTrained_models


In [42]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


In [43]:
emb_model = KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.bin', binary=True)


In [44]:
sentence_embeddings1 = []

for sentence in valEN_sen1[:100]:
    sentence_embedding = []
    for word in sentence:
        if word in emb_model:
            sentence_embedding.append(emb_model[word])
    if sentence_embedding:
        sentence_embedding = np.mean(sentence_embedding, axis=0)
    else:
        sentence_embedding = np.zeros(emb_model.vector_size)
    sentence_embeddings1.append(sentence_embedding)
# print(sentence_embeddings[0])
print((sentence_embeddings1[0].shape))


(300,)


In [45]:
sentence_embeddings2 = []

for sentence in outs:
    sentence_embedding = []
    for word in sentence:
        if word in emb_model:
            sentence_embedding.append(emb_model[word])
    if sentence_embedding:
        sentence_embedding = np.mean(sentence_embedding, axis=0)
    else:
        sentence_embedding = np.zeros(emb_model.vector_size)
    sentence_embeddings2.append(sentence_embedding)
# print(sentence_embeddings[0])
print((sentence_embeddings2[0].shape))


(300,)


In [46]:
from sklearn.metrics.pairwise import cosine_similarity
# sentence_embeddings1 and sentence_embeddings2 are the two lists of sentence embeddings
# of the same length
assert len(sentence_embeddings1) == len(sentence_embeddings2)

# Compute the cosine similarity between corresponding indexed sentences
cos_sim = []
for emb1, emb2 in zip(sentence_embeddings1, sentence_embeddings2):
    similarity = cosine_similarity(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]
    cos_sim.append(similarity)



In [48]:
sim_scores = test_score[:100]
# Normalize sim_scores using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
sim_scores_norm = scaler.fit_transform(np.array(sim_scores).reshape(-1, 1)).flatten()


# cos_sim and similarity_score are the two lists of similarity scores
# of the same length
assert len(cos_sim) == len(sim_scores)

# Calculate the Pearson correlation coefficient and its p-value
corr, p_value = pearsonr(cos_sim, sim_scores_norm)

print(f"Pearson correlation coefficient: {corr:.3f}")
print(f"P-value: {p_value:.3f}")



Pearson correlation coefficient: -0.055
P-value: 0.584
